<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7e374df463166d1ac6d741153567043ae93b0b820f909f259d1299f2aa1b9647
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-20 12:26:54
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 25.50 K (0.18%)
Current PnL: -20.95 L (-13.96%)
CY Booked + Current PnL: -9.05 L (-6.03%)
-------------------
Total profit:  1.96 L
Total loss:  -22.92 L
-------------------
Total Booked + Current PnL: 17.93 L (14.56%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.47%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.12 L (62.7%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 7.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-17.38,67.0,X-LC,3.99,220707.0,16207.0,7085.0,2.12,7.93,3.21,11.39,10.0,2.29,1.58,17.59,X40N,ATH,PHARMA
56,RELIANCE,1533.00,-8.78,73.0,X-LC,2.88,228181.0,17815.0,10976.0,3.24,8.47,4.81,13.68,37.0,1.62,1.64,26.39,XY25,NTT,REFINERIES
83,VOLTAS,1530.00,2.56,62.0,X-MC,1.55,215250.0,23508.0,14250.0,0.59,12.26,6.62,19.69,99.0,1.65,1.54,19.89,XY25,NTT,AC
76,TTKPRESTIG,770.00,95.52,47.0,M-SC,1.70,84233.0,-16544.0,16636.0,0.12,-16.42,19.75,0.09,245.0,-0.99,0.60,10.66,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-13.84,64.0,X-MC,2.64,148222.0,12089.0,19032.0,0.10,8.88,12.84,22.86,91.0,0.64,1.06,22.61,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNIONBANK,163.00,-10.08,60.0,M-LC,8.81,161750.0,20910.0,23583.0,4.08,14.85,14.58,31.59,66.0,0.89,1.16,45.27,XY24,NTT,BANKS
10,BANDHANBNK,400.00,-10.58,55.0,H-SC,2.91,223860.0,-54699.0,312128.0,3.78,-19.64,139.43,92.42,151.0,-0.18,1.61,29.37,XY24,NTT,BANKS
77,UJJIVANSFB,60.00,114.70,67.0,H-SC,12.87,135972.0,-6507.0,26025.0,3.37,-4.57,19.14,13.70,163.0,-0.25,0.98,60.48,OX40N,NTT,BANKS
45,KPIGREEN,731.05,4.41,64.0,H-SC,5.85,124526.0,-771.0,59698.0,3.28,-0.62,47.94,47.03,141.0,-0.01,0.89,55.92,MH,ATH,POWER
56,RELIANCE,1533.00,-8.78,73.0,X-LC,2.88,228181.0,17815.0,10976.0,3.24,8.47,4.81,13.68,37.0,1.62,1.64,26.39,XY25,NTT,REFINERIES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-26.90,37.0,H-SC,18.70,196100.0,-74661.0,404613.0,-5.00,-27.57,206.33,121.86,133.0,-0.18,1.41,53.41,AR,ATH,IT
31,HINDZINC,730.22,29.16,52.0,M-LC,10.08,210540.0,5464.0,107102.0,-3.25,2.66,50.87,54.89,52.0,0.05,1.51,27.40,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101250.0,9752.0,69002.0,-2.48,10.66,68.15,86.07,223.0,0.14,0.73,47.03,XR,NTT,DURABLES
67,STARHEALTH,761.00,20.77,65.0,H-SC,9.76,268818.0,-12305.0,145162.0,-1.94,-4.38,54.00,47.26,144.0,-0.08,1.93,44.03,XY24,NTT,INSURANCE
11,BANKINDIA,190.00,-26.71,53.0,H-MC,12.16,189343.0,9535.0,102870.0,-1.84,5.30,54.33,62.52,88.0,0.09,1.36,41.20,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,KPIGREEN,731.05,4.41,64.0,H-SC,5.85,124526.0,-771.0,59698.0,3.28,-0.62,47.94,47.03,141.0,-0.01,0.89,55.92,MH,ATH,POWER
36,INDIAMART,4810.62,-55.39,40.0,H-SC,7.76,124439.0,1103.0,130524.0,0.16,0.89,104.89,106.72,119.0,0.01,0.89,24.48,AR,ATH,MISC
85,WIPRO,420.00,-15.15,43.0,M-LC,6.00,149488.0,-1457.0,110905.0,0.09,-0.97,74.19,72.51,53.0,-0.01,1.07,5.00,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,114.70,67.0,H-SC,12.87,135972.0,-6507.0,26025.0,3.37,-4.57,19.14,13.70,163.0,-0.25,0.98,60.48,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,59.0,H-SC,1.91,227106.0,-42561.0,78897.0,-0.36,-15.78,34.74,13.47,138.0,-0.54,1.63,15.48,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,95.52,47.0,M-SC,1.70,84233.0,-16544.0,16636.0,0.12,-16.42,19.75,0.09,245.0,-0.99,0.60,10.66,OX40N,NTT,DURABLES
17,CERA,9475.0,-24.34,38.0,H-SC,2.49,137920.0,-37983.0,80007.0,-0.09,-21.59,58.01,23.89,149.0,-0.47,0.99,19.11,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.38,45.0,H-MC,7.52,102296.0,-29539.0,71761.0,0.27,-22.41,70.15,32.03,98.0,-0.41,0.73,14.58,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,52.0,M-LC,6.03,207515.0,4345.0,37788.0,0.63,2.14,18.21,20.74,55.0,0.11,1.49,7.80,XY25,NTT,FINANCE
31,HINDZINC,730.22,29.16,52.0,M-LC,10.08,210540.0,5464.0,107102.0,-3.25,2.66,50.87,54.89,52.0,0.05,1.51,27.40,X5K,ATH,METALS
78,UNIONBANK,163.00,-10.08,60.0,M-LC,8.81,161750.0,20910.0,23583.0,4.08,14.85,14.58,31.59,66.0,0.89,1.16,45.27,XY24,NTT,BANKS
11,BANKINDIA,190.00,-26.71,53.0,H-MC,12.16,189343.0,9535.0,102870.0,-1.84,5.30,54.33,62.52,88.0,0.09,1.36,41.20,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101250.0,9752.0,69002.0,-2.48,10.66,68.15,86.07,223.0,0.14,0.73,47.03,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101250.0,9752.0,69002.0,-2.48,10.66,68.15,86.07,223.0,0.14,0.73,47.03,XR,NTT,DURABLES
11,BANKINDIA,190.00,-26.71,53.0,H-MC,12.16,189343.0,9535.0,102870.0,-1.84,5.30,54.33,62.52,88.0,0.09,1.36,41.20,XR,NTT,BANKS
31,HINDZINC,730.22,29.16,52.0,M-LC,10.08,210540.0,5464.0,107102.0,-3.25,2.66,50.87,54.89,52.0,0.05,1.51,27.40,X5K,ATH,METALS
36,INDIAMART,4810.62,-55.39,40.0,H-SC,7.76,124439.0,1103.0,130524.0,0.16,0.89,104.89,106.72,119.0,0.01,0.89,24.48,AR,ATH,MISC
85,WIPRO,420.00,-15.15,43.0,M-LC,6.00,149488.0,-1457.0,110905.0,0.09,-0.97,74.19,72.51,53.0,-0.01,1.07,5.00,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,9.58,144964.0,-131559.0,241626.0,0.69,-47.58,166.68,39.81,54.0,-0.54,1.04,2.17,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,34.0,X-SC,38.78,51618.0,-13388.0,164316.0,0.68,-20.59,318.33,232.18,198.0,-0.08,0.37,0.68,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.45,39.0,X-MC,2.42,229351.0,-18380.0,93438.0,-1.43,-7.42,40.74,30.30,92.0,-0.20,1.64,4.89,X40,ATH,ELECTRICAL
55,RELAXO,1176.00,-40.07,41.0,X-SC,4.58,81648.0,-63512.0,139438.0,-0.90,-43.75,170.78,52.31,136.0,-0.46,0.59,9.29,X40N,NTT,FOOTWEAR
2,ACC,3906.00,-38.50,42.0,X-MC,3.23,182500.0,-55451.0,208105.0,-0.42,-23.30,114.03,64.15,174.0,-0.27,1.31,2.37,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.25,44.0,H-LC,0.36,155540.0,-30555.0,78034.0,-0.33,-16.42,50.17,25.51,15.0,-0.39,1.12,14.62,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,2.56,62.0,X-MC,1.55,215250.0,23508.0,14250.0,0.59,12.26,6.62,19.69,99.0,1.65,1.54,19.89,XY25,NTT,AC
28,HAVELLS,2069.16,-7.45,39.0,X-MC,2.42,229351.0,-18380.0,93438.0,-1.43,-7.42,40.74,30.30,92.0,-0.20,1.64,4.89,X40,ATH,ELECTRICAL
41,ITC,452.00,-37.50,59.0,X-LC,2.61,201590.0,1452.0,19433.0,0.02,0.73,9.64,10.44,4.0,0.07,1.45,6.33,X40,NTT,FMCG
33,ICICIGI,2260.25,-13.84,64.0,X-MC,2.64,148222.0,12089.0,19032.0,0.10,8.88,12.84,22.86,91.0,0.64,1.06,22.61,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,34.0,X-SC,38.78,51618.0,-13388.0,164316.0,0.68,-20.59,318.33,232.18,198.0,-0.08,0.37,0.68,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,9.58,144964.0,-131559.0,241626.0,0.69,-47.58,166.68,39.81,54.0,-0.54,1.04,2.17,XY24,NTT,AUTO
2,ACC,3906.00,-38.50,42.0,X-MC,3.23,182500.0,-55451.0,208105.0,-0.42,-23.30,114.03,64.15,174.0,-0.27,1.31,2.37,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-14.90,43.0,X-MC,5.45,179514.0,-20992.0,118138.0,0.03,-10.47,65.81,48.45,90.0,-0.18,1.29,3.52,X40N,ATH,FINANCE
73,TCS,4389.97,-29.01,50.0,X-LC,12.74,282658.0,-63298.0,129994.0,1.51,-18.30,45.99,19.28,1.0,-0.49,2.03,4.49,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,34.0,X-SC,38.78,51618.0,-13388.0,164316.0,0.68,-20.59,318.33,232.18,198.0,-0.08,0.37,0.68,XY24,NTT,MISC
55,RELAXO,1176.00,-40.07,41.0,X-SC,4.58,81648.0,-63512.0,139438.0,-0.90,-43.75,170.78,52.31,136.0,-0.46,0.59,9.29,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.27,96894.0,-31776.0,77069.0,1.49,-24.70,79.54,35.20,219.0,-0.41,0.69,11.21,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.27,46.0,X-SC,0.85,106453.0,4449.0,39473.0,0.16,4.36,37.08,43.06,122.0,0.11,0.76,22.22,X40N,ATH,MISC
32,HONAUT,58357.33,-22.75,48.0,X-SC,2.56,108855.0,-19083.0,66216.0,-0.07,-14.92,60.83,36.84,143.0,-0.29,0.78,11.44,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.01,50.0,X-LC,12.74,282658.0,-63298.0,129994.0,1.51,-18.30,45.99,19.28,1.0,-0.49,2.03,4.49,X40,ATH,IT
39,INFY,2275.00,-20.32,44.0,X-LC,7.63,310533.0,-2355.0,174054.0,1.17,-0.75,56.05,54.87,3.0,-0.01,2.23,5.85,X40,BTT,IT
41,ITC,452.00,-37.50,59.0,X-LC,2.61,201590.0,1452.0,19433.0,0.02,0.73,9.64,10.44,4.0,0.07,1.45,6.33,X40,NTT,FMCG
82,VBL,671.64,-19.14,50.0,X-LC,10.52,291406.0,-24436.0,137107.0,-1.04,-7.74,47.05,35.67,5.0,-0.18,2.09,5.05,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,51.0,H-LC,6.90,250368.0,-11251.0,130467.0,0.27,-4.30,52.11,45.57,7.0,-0.09,1.80,6.53,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7005.56,66.0,L-SC,6.46,77949.0,-15861.0,92479.0,3.08,-16.91,118.64,81.67,269.0,-0.17,0.56,50.95,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,51.0,L-SC,29.65,82291.0,-31258.0,71305.0,1.16,-27.53,86.65,35.27,268.0,-0.44,0.59,101.41,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,43.0,H-SC,9.42,90030.0,-7950.0,29557.0,0.91,-8.11,32.83,22.05,152.0,-0.27,0.65,30.87,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-40.03,63.0,M-SC,4.16,101250.0,9752.0,69002.0,-2.48,10.66,68.15,86.07,223.0,0.14,0.73,47.03,XR,NTT,DURABLES
45,KPIGREEN,731.05,4.41,64.0,H-SC,5.85,124526.0,-771.0,59698.0,3.28,-0.62,47.94,47.03,141.0,-0.01,0.89,55.92,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-187.50,70.0,M-SC,19.62,150732.0,1512.0,142668.0,0.43,1.01,94.65,96.62,208.0,0.01,1.08,63.12,XY25,NTT,FINANCE
77,UJJIVANSFB,60.0,114.70,67.0,H-SC,12.87,135972.0,-6507.0,26025.0,3.37,-4.57,19.14,13.70,163.0,-0.25,0.98,60.48,OX40N,NTT,BANKS
4,ANGELONE,3033.0,16.33,71.0,X-SC,8.27,206148.0,15142.0,42549.0,0.70,7.93,20.64,30.21,157.0,0.36,1.48,30.46,X40N,NTT,FINANCE
66,SONACOMS,1006.0,-34.84,65.0,M-SC,11.42,81148.0,-20011.0,94903.0,0.33,-19.78,116.95,74.03,202.0,-0.21,0.58,14.43,AR,BTT,AUTO
67,STARHEALTH,761.0,20.77,65.0,H-SC,9.76,268818.0,-12305.0,145162.0,-1.94,-4.38,54.00,47.26,144.0,-0.08,1.93,44.03,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,44.29
2,50,75.89


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.32
LC    33.98
MC    21.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.58
X40      14.83
XY25     12.46
XR       10.97
X40N     10.26
AR        7.99
OX40N     7.63
X200      1.81
X5K       1.51
SR        1.10
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.44
X-LC    22.98
X-MC    16.04
M-SC    12.72
M-LC     5.23
H-LC     4.73
X-SC     4.67
H-MC     3.78
M-MC     1.60
L-SC     1.49
L-LC     1.04
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.24,-3.48,37.59
IT,12.69,-18.90,83.08
FINANCE,12.19,-11.37,61.85
MISC,6.89,-19.43,85.61
BANKS,6.48,-10.73,68.97
PAINTS,5.69,-16.21,33.54
ELECTRICAL,5.34,-10.65,49.48
INSURANCE,3.95,-0.26,37.74
AUTO,2.81,-46.46,116.47


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3309678.0,22
XR,1382639.0,14
AR,1217459.0,9
X40,767793.0,10
X40N,597388.0,8
OX40N,583746.0,10
XY25,446639.0,8
SR,267166.0,2
X5K,107102.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3457330.0,25
M-SC,1265829.0,15
X-MC,1088473.0,11
X-LC,1035543.0,13
X-SC,529061.0,6
H-MC,352183.0,3
H-LC,280938.0,3
M-LC,279378.0,4
L-SC,253413.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1186682.0      6
           AR         846648.0      5
           XR         826392.0      7
M-SC       XY24       690344.0      6
X-MC       XY24       590253.0      4
X-LC       X40        472795.0      5
           XY24       306735.0      2
X-SC       X40N       287676.0      4
H-SC       OX40N      270744.0      4
           SR         267166.0      2
X-MC       X40        217929.0      4
H-LC       AR         208501.0      2
X-LC       X40N       191574.0      3
H-MC       XY24       177552.0      1
M-MC       XY24       170213.0      1
X-SC       XY24       164316.0      1
L-SC       XR         163784.0      2
M-SC       AR         162310.0      2
X-MC       XY25       162153.0      2
M-SC       OX40N      151612.0      4
           XY25       142668.0      1
           XR         118895.0      2
X-MC       X40N       118138.0      1
M-LC       XR         110905.0      1
           X5K        107102.0      1
H-MC       XR         102870.0      1
L-SC       OX40N       89629.0      1
X-SC       X40         77069.0      1
H-LC       X200        72437.0      1
H-MC       OX40N       71761.0      1
X-LC       XY25        64439.0      3
L-MC       XR          59793.0      1
H-SC       MH          59698.0      1
L-LC       XY25        39591.0      1
M-LC       XY25        37788.0      1
           XY24        23583.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
